In [ ]:
PATH = "C:/Users/magaxels/AutoML"
import sys; sys.path.insert(0, PATH)
from gazer import GazerMetaLearner
from gazer.ensembler import GazerMetaEnsembler
import numpy as np
import pandas as pd
import os
import time
from gazer.metrics import get_scorer
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import random


In [ ]:
X, y = load_digits(return_X_y=True)

In [ ]:
X.shape, y.shape

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

In [5]:
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
##scaler = StandardScaler()
scaler = MaxAbsScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [6]:
learner = GazerMetaLearner(method='all', exclude=['neuralnet', 'xgboost'], verbose=1)

C:\Users\magaxels\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Initialized: adaboost, knn, logreg, sgd_hinge, gaussian_nb, multinomial_nb, bernoulli_nb, random_forest, svm, tree


In [7]:
ensembler = GazerMetaEnsembler(learner, X_train.shape)

In [8]:
ensembler.summary()

Algorithm: logreg 	Fits: 200
Algorithm: adaboost 	Fits: 100
Algorithm: knn 	Fits: 60
Algorithm: sgd_hinge 	Fits: 200
Algorithm: gaussian_nb 	Fits: 1
Algorithm: svm 	Fits: 200
Algorithm: multinomial_nb 	Fits: 40
Algorithm: bernoulli_nb 	Fits: 40
Algorithm: random_forest 	Fits: 100
Algorithm: tree 	Fits: 120

Total number of fits = 1061


In [9]:
ensembler.fit(X_train, y_train, os.path.join(os.getcwd(), "models"))

tree: 100%|███████████████████████████████████████████████████████████████████████████| 120/120 [00:01<00:00, 84.10it/s]


In [12]:
valscores = {}
for n_best in range(3, 4):
    vs = ensembler.hillclimb(X_val, y_val, n_best=n_best, p=0.8, iterations=10, scoring='accuracy')
    valscores[str(n_best)] = vs

Max validation score =  0.9810901001112347
Algorithms in initial ensemble: 3
Initial accuracy-score: 0.9800
Iteration: 1 	Score: 0.981090
Iteration: 2 	Score: 0.983315
Iteration: 3 	Score: 0.983315
Iteration: 4 	Score: 0.983315
Iteration: 5 	Score: 0.983315
Iteration: 6 	Score: 0.983315
Iteration: 7 	Score: 0.983315
Iteration: 8 	Score: 0.983315
Iteration: 9 	Score: 0.983315
Iteration: 10 	Score: 0.983315


In [ ]:
valsscores = valscores['3']
x = [x for x,_ in valscores]
y = [y for _,y in valscores]
import matplotlib.pyplot as plt
% matplotlib inline
plt.figure(figsize=(9,9))
plt.plot(x,y)
plt.show()